<a href="https://colab.research.google.com/github/SFIComplexityExplorer/Mesa-ABM-Tutorial/blob/main/Session_13_Traders_Eat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Winati Mutmainnah (D121221014)

# Import Dependencies

In [38]:
import mesa
import numpy as np
import math
import matplotlib.pyplot as plt

# Helper Functions

In [39]:
def get_distance(pos_1, pos_2):
  '''
  Calculate the Euclidean distance between two positions

  used in trade.move()
  '''

  x1, y1 = pos_1
  x2, y2 = pos_2
  dx = x1 - x2
  dy = y1 - y2
  return math.sqrt(dx**2 + dy**2)

# Resource Classes

In [40]:
class Sugar(mesa.Agent):
  """
  Sugar:
  - contains an amount of sugar
  - grows 1 amount of sugar at each turn
  """

  def __init__(self, unique_id, model, pos, max_sugar):
    super().__init__(unique_id, model)
    self.pos = pos
    self.amount = max_sugar
    self.max_sugar = max_sugar


  def step(self):
    '''
    Sugar growth function, adds one unit of sugar each step until
    max amount
    '''
    self.amount = min([self.max_sugar, self.amount+1])

In [41]:
class Spice(mesa.Agent):
  """
  Spice:
  - contains an amount of spice
  - grows 1 amount of spice at each turn
  """

  def __init__(self, unique_id, model, pos, max_spice):
    super().__init__(unique_id, model)
    self.pos = pos
    self.amount = max_spice
    self.max_spice = max_spice

  def step(self):
    '''
    Spice growth function, adds one unit of spice each step until
    max amout
    '''
    self.amount = min([self.max_spice, self.amount+1])

# Trader Agent

In [42]:
class Trader(mesa.Agent):
  """
  Trader:
  - has a metabolism of sugar and spice
  - harvest and trade sugar and spice to survive
  """


  def __init__(self, unique_id, model, pos, moore=False, sugar=0,
               spice=0, metabolism_sugar=0, metabolism_spice=0,
               vision=0):
    super().__init__(unique_id, model)
    self.pos = pos
    self.moore = moore
    self.sugar = sugar
    self.spice = spice
    self.metabolism_sugar = metabolism_sugar
    self.metabolism_spice = metabolism_spice
    self.vision = vision


  def get_sugar(self, pos):
    '''
    used in self.get_sugar_amount()
    '''

    this_cell = self.model.grid.get_cell_list_contents(pos)
    for agent in this_cell:
      if type(agent) is Sugar:
        return agent
    return None


  def get_sugar_amount(self,pos):
    '''
    used in self.move() as part of self.calculate_welfare()
    '''

    sugar_patch = self.get_sugar(pos)
    if sugar_patch:
      return sugar_patch.amount
    return 0

  def get_spice(self, pos):
    '''
    used in self.get_spice_amount()
    '''

    this_cell = self.model.grid.get_cell_list_contents(pos)
    for agent in this_cell:
      if type(agent) is Spice:
        return agent
    return None

  def get_spice_amount(self, pos):
    '''
    used in self.move() as part of self.calculate_welfare()
    '''

    spice_patch = self.get_spice(pos)
    if spice_patch:
      return spice_patch.amount
    return 0


  def is_occupied_by_other(self,pos):
    '''
    helper function part 1 of self.move()
    '''

    if pos == self.pos:
      # agent's position is considered unoccupied as agent can stay there
      return False
    # get contents of each cell in neighborhood
    this_cell = self.model.grid.get_cell_list_contents(pos)
    for a in this_cell:
      # see if occupied by another agent
      if isinstance(a, Trader):
        return True
    return False

  def calculate_welfare(self, sugar, spice):
    '''
    helper function part 2 self.move()
    '''

    # calculate total resources
    m_total = self.metabolism_sugar + self.metabolism_spice
    # Cobb-Douglas functional form
    return sugar**(self.metabolism_sugar/m_total) * spice**(
        self.metabolism_spice/m_total)


  def is_starved(self):
    '''
    Helper function for self.maybe_die()
    '''

    return (self.sugar <= 0) or (self.spice <= 0)

  ######################################################################
  #                                                                    #
  #                      MAIN TRADE FUNCTIONS                          #
  #                                                                    #
  ######################################################################


  def move(self):
    '''
    Function for trader agent to identify optimal move for each step in 4 parts
    1 - identify all possible moves
    2 - determine which move maximizes welfare
    3 - find closest best option
    4 - move
    '''

    # 1. identify all possible moves

    neighbors = [i
                 for i in self.model.grid.get_neighborhood(
                   self.pos, self.moore, True, self.vision
                 ) if not self.is_occupied_by_other(i)]

    # 2. determine which move maximizes welfare

    welfares = [
        self.calculate_welfare(
            self.sugar + self.get_sugar_amount(pos),
            self.spice + self.get_spice_amount(pos))
        for pos in neighbors
    ]

    # 3. Find closest best option

    # find the highest welfare in welfares
    max_welfare = max(welfares)
    # get the index of max welfare cells
    candidate_indices = [i for i in range(len(welfares))
                        if math.isclose(welfares[i], max_welfare,
                                        rel_tol=1e-02)]

    # convert index to positions of those cells
    candidates = [neighbors[i] for i in candidate_indices]

    min_dist = min(get_distance(self.pos, pos) for pos in candidates)

    final_candidates = [ pos for pos in candidates
                       if math.isclose(get_distance(self.pos, pos), min_dist, rel_tol=1e-02
                                       )]
    self.random.shuffle(final_candidates)

    # 4. Move Agent
    self.model.grid.move_agent(self, final_candidates[0])

  def eat(self):
    # get sugar
    sugar_patch = self.get_sugar(self.pos)

    if sugar_patch:
        self.sugar += sugar_patch.amount
        sugar_patch.amount = 0
    self.sugar -= self.metabolism_sugar

    # get_spice
    spice_patch = self.get_spice(self.pos)

    if spice_patch:
        self.spice += spice_patch.amount
        spice_patch.amount = 0
    self.spice -= self.metabolism_spice

  def maybe_die(self):
    '''
    Function to remove Traders who have consumed all their sugar or spice
    '''

    if self.is_starved():
      print(self.unique_id, self.model.schedule.get_type_count(Trader))
      self.model.grid.remove_agent(self)
      self.model.schedule.remove(self)
      print(self.unique_id, self.model.schedule.get_type_count(Trader))



# Model Class

In [43]:
class SugarscapeG1mt(mesa.Model):
  """
  Manager class to run Sugarscape with Traders
  """


  def __init__(self, width=50,height=50, initial_population=200,
               endowment_min=25, endowment_max=50, metabolism_min=1,
               metabolism_max=5, vision_min=1, vision_max=5):
    super().__init__()

    #Initiate width and heigh of sugarscape
    self.width = width
    self.height = height
    #Initiate population attributes
    self.initial_population = initial_population
    self.endowment_min = endowment_min
    self.endowment_max = endowment_max
    self.metabolism_min = metabolism_min
    self.metabolism_max = metabolism_max
    self.vision_min = vision_min
    self.vision_max = vision_max

    #initiate activation schedule
    self.schedule = mesa.time.RandomActivationByType(self)
    #initiate mesa grid class
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

    #read in landscape file from supplmentary material
    sugar_distribution =np.genfromtxt("sugar-map.txt")
    spice_distribution = np.flip(sugar_distribution, 1)

    agent_id = 0
    for _,(x,y) in self.grid.coord_iter():
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0:
        sugar = Sugar(agent_id, self, (x,y), max_sugar)
        self.schedule.add(sugar)
        self.grid.place_agent(sugar, (x,y))
        agent_id += 1

      max_spice = spice_distribution[x,y]
      if max_spice > 0:
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.schedule.add(spice)
        self.grid.place_agent(spice, (x,y))
        agent_id += 1

    for i in range(self.initial_population):
      #get agent position
      x = self.random.randrange(self.width)
      y = self.random.randrange(self.height)
      #see Growing Artificial Societies p. 108 for initialization
      #give agents initial endowment
      sugar = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
      spice = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
      #give agents initial metabolism
      metabolism_sugar = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
      metabolism_spice = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
      #give agents vision
      vision = int(self.random.uniform(self.vision_min, self.vision_max+1))
      #create Trader object
      trader = Trader(agent_id,
                      self,
                      (x,y),
                      moore = False,
                      sugar = sugar,
                      spice = spice,
                      metabolism_sugar = metabolism_sugar,
                      metabolism_spice = metabolism_spice,
                      vision = vision)
      #place agent
      self.grid.place_agent(trader, (x,y))
      self.schedule.add(trader)
      agent_id += 1

  def step(self):
    '''
    Unique step function that does staged activation of sugar and spice
    and then randomly activates traders
    '''
    # step Sugar agents
    for sugar in self.schedule.agents_by_type[Sugar].values():
      sugar.step()

    # step Spice agents
    for spice in self.schedule.agents_by_type[Spice].values():
      spice.step()

    # step trader agents
    # to account for agent death and removal we need a seperate data strcuture to
    # iterate
    trader_shuffle = list(self.schedule.agents_by_type[Trader].values())
    self.random.shuffle(trader_shuffle)

    for agent in trader_shuffle:
      agent.move()
      agent.eat()
      agent.maybe_die()

    self.schedule.steps += 1 #important for data collector to track number of steps

  def run_model(self, step_count=1000):

    for i in range(step_count):
      self.step()

# Run Sugarscape

In [44]:
model = SugarscapeG1mt()
model.run_model(step_count=15)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

4277 200
4277 199
4326 199
4326 198
4155 198
4155 197
4191 197
4191 196
4328 196
4328 195
4275 195
4275 194
4331 194
4331 193
4329 193
4329 192
4288 192
4288 191
4295 191
4295 190
4312 190
4312 189
4265 189
4265 188
4217 188
4217 187
4272 187
4272 186
4324 186
4324 185
4280 185
4280 184
4239 184
4239 183
4225 183
4225 182
4261 182
4261 181
4297 181
4297 180
4168 180
4168 179
4296 179
4296 178
4253 178
4253 177
4320 177
4320 176
4283 176
4283 175
4211 175
4211 174
4210 174
4210 173
4187 173
4187 172
4321 172
4321 171
4177 171
4177 170
4203 170
4203 169
4233 169
4233 168
4143 168
4143 167
4157 167
4157 166
4161 166
4161 165
4142 165
4142 164
4300 164
4300 163
4204 163
4204 162
4205 162
4205 161
4255 161
4255 160
4325 160
4325 159
4167 159
4167 158
4201 158
4201 157
4246 157
4246 156
4242 156
4242 155
4250 155
4250 154
4235 154
4235 153
4156 153
4156 152
4147 152
4147 151
4196 151
4196 150
4159 150
4159 149
4180 149
4180 148
4141 148
4141 147
4148 147
4148 146
4164 146
4164 145
4274 145
4